In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

''' 
        This is the classical representation of the Quantum Tiq Taq Toe game board
        in the form of a Python list within list structure. Each sublist represents
        one square on the actual Tic Tac Toe game baord.
'''

BOARD = [[""], [""], [""], 
        [""], [""], [""],
        [""], [""], [""]]

In [2]:
# The number of qubits that have been used to represent our game board
QUBITS = 18

'''
    This function is used to initialize a QuantumCircuit with the desired
    number of quantum and classical bits, the value of which is taken in
    as a parameter. It returns the initialized QuantumCircuit.
'''
def initialize_qubit_state(QUBITS):
    initial_state = QuantumCircuit(QUBITS, QUBITS)
    return initial_state

'''
    This function executes the QuantumCircuit in the parameter and returns
    the result of the counts.
'''
def collapse(QCIRCUIT):
    job = execute(QCIRCUIT, Aer.get_backend('qasm_simulator'), shots=1)
    counts = job.result().get_counts(QCIRCUIT)
    return counts

'''
    This function measures the output on the classical bits of the
    QuantumCircuit passed to it as a parameter.
'''
def measurement(QCIRCUIT):
    QCIRCUIT.measure(range(18), range(17, -1, -1))
    return QCIRCUIT

In [3]:
'''
    This function marks the superposition move X using a particular sequence of gates. 
    It takes as parameter the temporary QuantumCircuit, and two indexes where the move is to be marked.
'''
def superposition_move_x(QCIRCUIT, INDEX1, INDEX2):
    QCIRCUIT.h(INDEX1*2)
    QCIRCUIT.cx(INDEX1*2, INDEX2*2)
    QCIRCUIT.x(INDEX1*2)
    QCIRCUIT.cx(INDEX1*2, INDEX1*2 + 1)
    QCIRCUIT.cx(INDEX2*2, INDEX2*2 + 1)
    return QCIRCUIT

''' 
    This function marks the superposition move O using a particular sequence of gates.
    It takes as parameter the temporary QuantumCircuit, and two indexes where the move is to marked.
'''
def superposition_move_o(QCIRCUIT, INDEX1, INDEX2):
    QCIRCUIT.h(INDEX1*2)
    QCIRCUIT.cx(INDEX1*2, INDEX2*2)
    QCIRCUIT.x(INDEX1*2)
    return QCIRCUIT

'''
    This function marks the classical move X using a particular sequence of gates.
    It takes as parameter the original QuantumCircuit, and an index where the move is to marked.
'''
def classic_move_x(QCIRCUIT, INDEX):
    QCIRCUIT.x(INDEX*2)
    QCIRCUIT.x(INDEX*2 + 1)
    return QCIRCUIT

'''
    This function marks the classical move O using a particular sequence of gates.
    It takes as parameter the original QuantumCircuit, and an index where the move is to marked.
'''
def classic_move_o(QCIRCUIT, INDEX):
    QCIRCUIT.x(INDEX*2)
    return QCIRCUIT

'''
    This function updates the original state. 
    It takes as parameter the original QuantumCircuit and a state list that came as a result when 
    the qubit state in superposition was collapsed.
    It uses the state list to alter the original 18 qubit state.
'''
def update_original_state(QCIRCUIT, STATE):
    state_list = list(STATE.keys())

    for i in range(9):
        if (state_list[0][i*2:i*2 + 2] == "11"):
            QCIRCUIT.x(i*2)
            QCIRCUIT.x(i*2 + 1)
        elif (state_list[0][i*2:i*2 + 2] == "10"):
            QCIRCUIT.x(i*2)
    return QCIRCUIT

In [4]:
'''
    This function updates the 3 x 3 board whenever a classical move is made.
    It takes as input the original QuantumCircuit and the current game board.
'''
def board_update_classic(QCIRCUIT, BOARD):
    current_state = QCIRCUIT.copy()
    current_state = measurement(current_state)
    current_state = collapse(current_state)
    current_state_list = list(current_state.keys())

    for i in range(9):
        if (current_state_list[0][i*2:i*2 + 2] == "11"):
            BOARD[i] = "X"
        elif (current_state_list[0][i*2:i*2 + 2] == "10"):
            BOARD[i] = "O"
    return BOARD   

'''
    This function updates the 3 x 3 board whenever a superposition move is made.
    It resets the board and then updates it classically.
'''
def board_update_superposition(QCIRCUIT, BOARD):
    BOARD = reset_board()
    board_update_classic(QCIRCUIT, BOARD)
    return BOARD

'''
    This function is called after every move and it checks for the winner. 
    If a winner is found, that is, if one player manages to mark 3 Xs or Os
    in a row (vertical, horizontal, or diagonal), the function returns true. 
    It takes as parameter a board and a player. Returns whether the player won
    or not. 
'''
def winning(BOARD, PLAYER):
    win = False

    # check for player = 1
    if (PLAYER == 1):
        # horizontal
        if ((BOARD[0] == "X" and BOARD[1] == "X" and BOARD[2] == "X")):
            win = True
        elif ((BOARD[3] == "X" and BOARD[4] == "X" and BOARD[5] == "X")):
            win = True
        elif ((BOARD[6] == "X" and BOARD[7] == "X" and BOARD[8] == "X")):
            win = True
        # vertical
        elif ((BOARD[0] == "X" and BOARD[3] == "X" and BOARD[6] == "X")):
            win = True
        elif ((BOARD[1] == "X" and BOARD[4] == "X" and BOARD[7] == "X")):
            win = True
        elif ((BOARD[2] == "X" and BOARD[5] == "X" and BOARD[8] == "X")):
            win = True
        # diagonal
        elif ((BOARD[0] == "X" and BOARD[4] == "X" and BOARD[8] == "X")):
            win = True
        elif ((BOARD[2] == "X" and BOARD[4] == "X" and BOARD[6] == "X")):
            win = True

    # check for player = 2
    if (PLAYER == 2):
        # horizontal
        if ((BOARD[0] == "O" and BOARD[1] == "O" and BOARD[2] == "O")):
            win = True
        elif ((BOARD[3] == "O" and BOARD[4] == "O" and BOARD[5] == "O")):
            win = True
        elif ((BOARD[6] == "O" and BOARD[7] == "O" and BOARD[8] == "O")):
            win = True
        # vertical
        elif ((BOARD[0] == "O" and BOARD[3] == "O" and BOARD[6] == "O")):
            win = True
        elif ((BOARD[1] == "O" and BOARD[4] == "O" and BOARD[7] == "O")):
            win = True
        elif ((BOARD[2] == "O" and BOARD[5] == "O" and BOARD[8] == "O")):
            win = True
        # diagonal
        elif ((BOARD[0] == "O" and BOARD[4] == "O" and BOARD[8] == "O")):
            win = True
        elif ((BOARD[2] == "O" and BOARD[4] == "O" and BOARD[6] == "O")):
            win = True

    return win

In [5]:
'''
    This function checks whether or not the input index is occupied.
    Returns true if occupied, else false.
'''
def occupied(BOARD, INDEX):
    if (BOARD[INDEX][0] != ""):
        return True
    return False

'''
    This function resets the classical board and makes it empty again.
'''
def reset_board():
    BOARD = [[""], [""], [""], 
        [""], [""], [""],
        [""], [""], [""]]
    return BOARD

'''
    This function is used to print the game board as an output.
'''
def print_board(BOARD):
    print(BOARD[0:3])
    print(BOARD[3:6])
    print(BOARD[6:9])
    print()

'''
    This function checks whether or not the game board is full or not.
'''
def board_full(BOARD):
    full = True
    for i in range(9):
        if (BOARD[i][0] == ""):
            full = False
    return full

In [6]:
#################### MAIN PROGRAM ####################

count = 0
ttt = initialize_qubit_state(QUBITS)
temp = QuantumCircuit(18, 18)
BOARD = reset_board()

# main loop
while (True):

    # player 1's turn 
    if count % 2 == 0:

        # prompt for an input. 
        move = input("What move would Player One like to make? Enter c for classic move and s for superposition move: ")
        
        # if classical move, then check if it's a valid move and mark it in the given index. 
        if (move == "c"):
            index_one = int(input("Player One's turn! Please enter the index (0-8) at which you want to place an X: "))
            
            # checking move validity.
            if (index_one > 8):
                print("Please enter a valid index!")
                index_one = int(input("Player One's turn! Please enter the index (0-8) at which you want to place an X: "))
            
            # checking occupancy.
            if (occupied(BOARD, index_one) == True):
                print("This location is already occupied on the board. Enter a new index")
                index_one = int(input("Player One's turn! Please enter the index (0-8) at which you want to place an X: "))
            
            # marking the move.
            elif (occupied(BOARD, index_one) == False):
                ttt = classic_move_x(ttt, index_one)
                BOARD = board_update_classic(ttt, BOARD)
                count += 1
                
                # if the board is full, collapse all the superposition states and check for a winner. 
                if (board_full(BOARD) == True):
                    print_board(BOARD)
                    temp = measurement(temp)
                    state = collapse(temp)
                    ttt = update_original_state(ttt, state)
                    BOARD = board_update_superposition(ttt, BOARD)
                    temp = QuantumCircuit(18, 18)

                    if (winning(BOARD, 1) == True):
                        print("Player one has won!")
                        break

                if (board_full(BOARD) == False):
                    if (winning(BOARD, 1) == True):
                        print("Player one has won!")
                        break
                    if (winning(BOARD, 2) == True):
                        print("Player two has won!")
                        break

        # if superposition move, then check if it's a valid move and mark it in the given index. 
        elif (move == "s"):
            index_one_1 = int(input("Player One's turn! Please enter the first index (0-8) at which you want to place an X: "))

            # checking occupancy.
            if (occupied(BOARD, index_one_1) == True or (index_one_1 > 8)):
                print("This location is already occupied on the board or the index is invalid. Enter a new index!")
                index_one_1 = int(input("Player One's turn! Please enter the first index (0-8) at which you want to place an X: "))
            
            elif (occupied(BOARD, index_one_1) == False and (index_one_1 <= 8)):
                index_one_2 = int(input("Player One's turn! Please enter the second index (0-8) at which you want to place an X: "))

                if (occupied(BOARD, index_one_2) == True or (index_one_2 > 8)):
                    print("This location is already occupied on the board. Enter a new index!")
                    index_one_2 = int(input("Player One's turn! Please enter the second index (0-8) at which you want to place an X: "))

                # marking the move on the given indexes.
                elif (occupied(BOARD, index_one_2) == False or (index_one_2 <= 8)):
                    temp = superposition_move_x(temp, index_one_1, index_one_2)
                    BOARD[index_one_1] = "X_" + str(count)
                    BOARD[index_one_2] = "X_" + str(count)
                    count += 1

                    # if the board is full, collapse all the superposition states and check for a winner. 
                    if (board_full(BOARD) == True):
                        print_board(BOARD)
                        temp = measurement(temp)
                        state = collapse(temp)
                        ttt = update_original_state(ttt, state)
                        BOARD = board_update_superposition(ttt, BOARD)
                        temp = QuantumCircuit(18, 18)
                        
                        if ((winning(BOARD, 1) == True) and (winning(BOARD, 2) == True)):
                            print("We have a tie!")

                        if (winning(BOARD, 1) == True):
                            print("Player one has won!")
                            break
        
                        if (board_full(BOARD) == False):
                            if (winning(BOARD, 1) == True):
                                print("Player one has won!")
                                break
                            if (winning(BOARD, 2) == True):
                                print("Player two has won!")
                                break

        else:
            print("Please enter a valid move!")
            move = input("What move would Player One like to make? Enter c for classic move and s for superposition move: ")

    print_board(BOARD)

    # player 2's turn 
    if count % 2 != 0:

        # prompt for an input
        move = input("What move would Player Two like to make? Enter c for classic move and s for superposition move: ")
        
        # if classical move, then check if it's a valid move and mark it in the given index. 
        if (move == "c"):
            index_two = int(input("Player Two's turn! Please enter the index (0-8) at which you want to place an O: "))
            
            # checking move validity.
            if (index_two > 8):
                print("Please enter a valid index!")
                index_two = int(input("Player Two's turn! Please enter the index (0-8) at which you want to place an O: "))

            # checking occupancy.
            if (occupied(BOARD, index_two) == True):
                print("This location is already occupied on the board. Enter a new index!")
                index_two = int(input("Player Two's turn! Please enter the index (0-8) at which you want to place an O: "))
            
            # marking the move.
            else:
                ttt = classic_move_o(ttt, index_two)
                BOARD = board_update_classic(ttt, BOARD)
                count += 1
                
                # if the board is full, collapse all the superposition states and check for a winner. 
                if (board_full(BOARD) == True):
                    temp = measurement(temp)
                    state = collapse(temp)
                    ttt = update_original_state(ttt, state)
                    BOARD = board_update_superposition(ttt, BOARD)
                    temp = QuantumCircuit(18, 18)

                    if (winning(BOARD, 2) == True):
                        print("Player two has won!")
                        break
                
                if (board_full(BOARD) == False):
                    if (winning(BOARD, 2) == True):
                        print("Player two has won!")
                        break
                    if (winning(BOARD, 1) == True):
                        print("Player one has won!")
                        break
        
        # if superposition move, then check if it's a valid move and mark it in the given index. 
        elif (move == "s"):
            index_two_1 = int(input("Player Two's turn! Please enter the first index (0-8) at which you want to place an O: "))

            # checking occupancy.
            if (occupied(BOARD, index_two_1) == True or (index_two_1 > 8)):
                print("This location is already occupied on the board or the index is invalid. Enter a new index!")
                index_two_1 = int(input("Player Two's turn! Please enter the first index (0-8) at which you want to place an O: "))
            
            elif (occupied(BOARD, index_two_1) == False and (index_two_1 <= 8)):
                index_two_2 = int(input("Player Two's turn! Please enter the second index (0-8) at which you want to place an O: "))

                if (occupied(BOARD, index_two_2) == True or (index_two_2 > 8)):
                    print("This location is already occupied on the board. Enter a new index!")
                    index_two_2 = int(input("Player Two's turn! Please enter the second index (0-8) at which you want to place an O: "))

                # marking the move on the given indexes.
                elif (occupied(BOARD, index_two_2) == False and (index_two_2 <= 8)):
                    temp = superposition_move_o(temp, index_two_1, index_two_2)
                    BOARD[index_two_1] = "O_" + str(count)
                    BOARD[index_two_2] = "O_" + str(count)
                    count += 1

                    # if the board is full, collapse all the superposition states and check for a winner. 
                    if (board_full(BOARD) == True):
                        print_board(BOARD)
                        temp = measurement(temp)
                        state = collapse(temp)
                        ttt = update_original_state(ttt, state)
                        BOARD = board_update_superposition(ttt, BOARD)
                        temp = QuantumCircuit(18, 18)

                        if ((winning(BOARD, 1) == True) and (winning(BOARD, 2) == True)):
                            print("We have a tie!")

                        if (winning(BOARD, 2) == True):
                            print("Player two has won!")
                            break

                        if (board_full(BOARD) == False):
                            if (winning(BOARD, 2) == True):
                                print("Player two has won!")
                                break
                            if (winning(BOARD, 1) == True):
                                print("Player one has won!")
                                break

        else:
            print("Please enter a valid move!")
            move = input("What move would Player Two like to make? Enter c for classic move and s for superposition move: ")
    
    print_board(BOARD)

print_board(BOARD)

['X_0', 'X_0', ['']]
[[''], [''], ['']]
[[''], [''], ['']]

['X_0', 'X_0', 'O_1']
[[''], [''], ['']]
['O_1', [''], ['']]

['X_0', 'X_0', 'O_1']
[[''], 'X_2', ['']]
['O_1', 'X_2', ['']]

['X_0', 'X_0', 'O_1']
[[''], 'X_2', 'O_3']
['O_1', 'X_2', 'O_3']

['X_0', 'X_0', 'O_1']
['X', 'X_2', 'O_3']
['O_1', 'X_2', 'O_3']

[[''], 'X', ['']]
['X', [''], 'O']
['O', 'X', ['']]

[[''], 'X', 'O_5']
['X', [''], 'O']
['O', 'X', 'O_5']

['X_6', 'X', 'O_5']
['X', 'X_6', 'O']
['O', 'X', 'O_5']

Player one has won!
[[''], 'X', ['']]
['X', 'X', 'O']
['O', 'X', 'O']

